In [9]:
import ccxt
import joblib
import time
import numpy as np
from Indicator import DataManage
from DataScaler import Data_StandardScaler
from sklearn.preprocessing import StandardScaler
from DB_Manage import DB_Bot
import pandas as pd
from keras.models import load_model
def download_data(frame):
  binance = ccxt.binance()
  btc_ohlcv = binance.fetch_ohlcv("BTC/USDT",limit = 500,timeframe=frame)
  df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
  df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
  df.set_index('datetime', inplace=True)
  return df
model = load_model("DNN_Model.h5")
trade_history = []
api_key = "yqrURCKivzwjsTyzxs16JIotlcVVUbHKq71uQQcqIYACzeMwU65BY3HDgqnB2ijL"
secret  = "gyfUDaf559JJA8qmqQE8ZK3pOUd7vad26ZUEYOalpUVy5ScBBWnJOFNZSkyeUMjF"

binance = ccxt.binance(config={
    'apiKey': api_key, 
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})
symbol = "ETC_USDT_1m"
temp = symbol.split('_')
timeframe = temp[2]
name = temp[0] + "/" + temp[1]
# 레버리지 변경
markets = binance.load_markets()
market = binance.market(name)
leverage = 10
resp = binance.fapiPrivate_post_leverage({
    'symbol': market['id'],
    'leverage': leverage
})

In [23]:
def buy(Account,price):
  Account["average_price"] = ( ( Account["amount"] * Account["average_price"] ) + ( price * 0.01 ) ) / ( Account["amount"] + 0.01 )
  Account["amount"] += 0.01
  return Account
def sell(Account,price):
  Account["result"] += (price - Account["average_price"]) * Account["amount"]
  Account["amount"] = 0
  Account["average_price"] = 0
  return Account
def trade():
  Account = {
    "result" : 0,
    "amount" : 0,
    "average_price" : 0
  }
  previous_time = '2018-01-01 00:00:00'
  previous_state = -1
  parameter = [
            {"rsi" : {"period" : 14}},
            {"ma" : {"period" : 7}},
            {"ma" : {"period" : 25}},
            {"ema" :{"period" : 7}},
            {"ema" :{"period" : 25}},
            {"stochastic" : {"n" : 14,"m" : 5,"t" : 5}},
            {"bb" : {"length" : 21,"std" : 2}},
            {"kdj" : {}},
            {"macd" : {"fast_period": 12, "slow_period" : 26}}
  ]

  print("-----자동매매 준비중-----")

  print(">> 데이터를 준비하는 중...")
  data = DB_Bot(symbol).GetData()
  data = data.drop(["datetime"],axis = 1).dropna()
  print(">> 보조지표 생성중...")
  DataManageBot = DataManage(data, parameter = parameter)
  data = DataManageBot.get_data()
  print(">> 데이터를 정규화하는 중...")
  scaler = StandardScaler()
  scaler.fit(data)
  
  print("-----자동매매 시작!-----")
  while(True):
    data = download_data(timeframe)
    DataManageBot = DataManage(data, parameter = parameter)
    data = DataManageBot.get_data()


    x = data.iloc[-2:]
    x = scaler.transform(x)
    pred = model.predict(x)

    now_time = data.index[-1]
    now_state = pred[-1]

    # 이전과 시간이 다르므로 조건 만족시 구매 
    if now_time != previous_time and int(pred[-2][0]) == 1:
      # order = binance.create_market_buy_order(
      # symbol="BTC/USDT",
      # amount=0.01)
      Account = buy(Account,data.iloc[-1]['open'])
      previous_time = now_time
      previous_state = now_state

    elif now_time != previous_time and int(pred[-2][0]) == 0 and Account["amount"] >= 0.01:
      # order = binance.create_market_sell_order(
      # symbol="BTC/USDT",
      # amount=Account["amount"])
      print("sell")
      Account = sell(Account,data.iloc[-1]['open'])

    elif now_time != previous_time:
      previous_time = now_time
      previous_state = now_state

    print("==============================================")
    print("현재 time:",now_time)
    print("현재 가격:",data.iloc[-1]['close'])
    print("현재 수량:",Account["amount"])
    print("현재 평단가:",Account["average_price"])
    print("현재 수익률:",round((data.iloc[-1]['close'] - Account["average_price"])/data.iloc[-1]['close'] * 100,2),"%")
    print("현재 추세 예측(1이면 다음 시가에 사야하고 0이면 팔아야함):",int(pred[-2][0]))
    print("수익:",Account["result"])
    
    line = []
    line.append(now_time)
    line.append(now_state)
    line.append(Account["result"])
    line.append(round((data.iloc[-1]['close'] - Account["average_price"])/data.iloc[-1]['close'] * 100,2))
    trade_history.append(line)
    time.sleep(5)
trade()

-----자동매매 준비중-----
>> 데이터를 준비하는 중...
>> 보조지표 생성중...
>> 데이터를 정규화하는 중...
-----자동매매 시작!-----
현재 time: 2023-05-16 10:12:00
현재 가격: 27076.17
현재 수량: 0.01
현재 평단가: 27076.909999999996
현재 수익률: -0.0 %
현재 추세 예측(1이면 다음 시가에 사야하고 0이면 팔아야함): 1
수익: 0
현재 time: 2023-05-16 10:12:00
현재 가격: 27078.26
현재 수량: 0.01
현재 평단가: 27076.909999999996
현재 수익률: 0.0 %
현재 추세 예측(1이면 다음 시가에 사야하고 0이면 팔아야함): 1
수익: 0
현재 time: 2023-05-16 10:12:00
현재 가격: 27078.27
현재 수량: 0.01
현재 평단가: 27076.909999999996
현재 수익률: 0.01 %
현재 추세 예측(1이면 다음 시가에 사야하고 0이면 팔아야함): 1
수익: 0
현재 time: 2023-05-16 10:12:00
현재 가격: 27078.26
현재 수량: 0.01
현재 평단가: 27076.909999999996
현재 수익률: 0.0 %
현재 추세 예측(1이면 다음 시가에 사야하고 0이면 팔아야함): 1
수익: 0
현재 time: 2023-05-16 10:12:00
현재 가격: 27078.26
현재 수량: 0.01
현재 평단가: 27076.909999999996
현재 수익률: 0.0 %
현재 추세 예측(1이면 다음 시가에 사야하고 0이면 팔아야함): 1
수익: 0
현재 time: 2023-05-16 10:13:00
현재 가격: 27090.91
현재 수량: 0.02
현재 평단가: 27083.454999999998
현재 수익률: 0.03 %
현재 추세 예측(1이면 다음 시가에 사야하고 0이면 팔아야함): 1
수익: 0
현재 time: 2023-05-16 10:13:00
현재 가격: 27091.16
현재 수량: 0.0

KeyboardInterrupt: 

In [22]:
[1.] == 1

False

In [ ]:
parameter = [
        {"rsi" : {"period" : 14}},
        {"ma" : {"period" : 7}},
        {"ma" : {"period" : 25}},
        {"ema" :{"period" : 7}},
        {"ema" :{"period" : 25}},
        {"stochastic" : {"n" : 14,"m" : 5,"t" : 5}},
        {"bb" : {"length" : 21,"std" : 2}},
        {"kdj" : {}},
        {"macd" : {"fast_period": 12, "slow_period" : 26}}
]
print(">> 자동매매 준비중...")
print(">> 데이터를 준비하는 중...")
data = DB_Bot(symbol).GetData()
data = data.drop(["datetime"],axis = 1).dropna()
print(">> 보조지표 생성중...")
DataManageBot = DataManage(data, parameter = parameter)
print(">> 데이터를 정규화하는 중...")
scaler = StandardScaler()
scaler.fit(data)

>> 자동매매 준비중...
>> 데이터를 준비하는 중...
>> 보조지표 생성중...
>> 데이터를 정규화하는 중...


StandardScaler()